# **Brand Expansion - Colombia**
## BeerTrotters_COL


Brand Expansion esta relacionado con el proceso de introduccir un producto a un mercado o area. En este reto se busca determinar la probalidad que un cliente compre un determinado producto en el siguiente mes, conocer esta información es de suma importancia para el negocio ya que permite crear estrategias para impulsar, mantener o frenar el producto, con el fin de matener el status de la marca.

# **Imports** 

In [1]:
#Imports-preparacion 

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sb
import dateutil.relativedelta
from scipy.stats import linregress
import math
%matplotlib inline

#Imports-Machine learning 
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
import multiprocessing
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingClassifier

# **Definicion de funciones** 

In [2]:
#Funcion para calculo de pendiente.

def get_slope(array):
  '''
    retorna la pendiente de un array indicado 

    Parameters:
        array: array de valores para calcular la pendiente.

    Returns:
        slope: pendiente.
    '''

  y = np.array(array)
  x = np.arange(len(y))
  slope, intercept, r_value, p_value, std_err = linregress(x,y)
  return slope

In [3]:
def base_slope(df,fecha_prediccion,periodo):

    '''
    retorna la pendiente del volumen de compra por cliente dado un perido y fecha de predicción.

    Parameters:
        df: Base de datos donde se encuentra la variable Volumen.
        fecha_prediccion: Fecha corte del mes que se desea predecir (Ejemplo : '2020-08-01')
        periodo : Meses que se desean agrupar.

    Returns:
        slope: pendiente de la variable volumen por cliente en el periodo dado.
    '''

  fecha_fin=pd.to_datetime(fecha_prediccion) - dateutil.relativedelta.relativedelta(months=1)  
  fecha_inicio = pd.to_datetime(fecha_prediccion) - dateutil.relativedelta.relativedelta(months=periodo)  
  df= df[(df['Fecha']<=pd.to_datetime(fecha_fin))&(df['Fecha']>=pd.to_datetime(fecha_inicio))]
  df=df[['Cliente','Fecha','volumen_cliente']]
  df.sort_values(['Cliente','Fecha'],inplace=True)
  slope=df.groupby(['Cliente'])['volumen_cliente'].apply(get_slope)
  slope=slope.reset_index()
  slope.columns = ['Cliente',	'slope']
  slope.fillna(0, inplace=True)

  return slope

In [4]:
def base_periodo(df,fecha_prediccion,meses, estadisticas):

      '''
    retorna la Base agrupada por meses indicados , usando estadisticas definidas para la agrupación. 

    Parameters:
        df: Base para agrupar.
        fecha_prediccion: Fecha corte del mes que se desea predecir (Ejemplo : '2020-08-01')
        periodo : Meses que se desean agrupar.
        estadisticas : Lista de estadisticas

    Returns:
        base_perido_final: Base agrupada de variables por las estadisticas indicadas y el periodo dado.
    '''

  fecha_fin=pd.to_datetime(fecha_prediccion) - dateutil.relativedelta.relativedelta(months=1)  
  fecha_inicio = pd.to_datetime(fecha_prediccion) - dateutil.relativedelta.relativedelta(months=meses)  
  df= df[(df['Fecha']<=pd.to_datetime(fecha_fin))&(df['Fecha']>=pd.to_datetime(fecha_inicio))]
  
  columns = df.columns.to_list()
  columns.remove('Cliente')
  columns.remove('Fecha') 
  
  agg_group ={}
  for i in columns :
    key=i
    agg_group[key] = estadisticas
  
  agg_group['Fecha'] =['nunique']

  base_periodo = df.groupby(['Cliente']).agg(agg_group).reset_index()
  periodo_str='_'+str(meses)+'_'
  base_periodo.columns = [periodo_str.join(col) for col in base_periodo.columns]
  columns_name={}
  name='Cliente'+periodo_str
  columns_name[name] ='Cliente'
  base_periodo.rename(columns=columns_name, inplace=True)

  slope = base_slope(grouped_cliente,fecha_prediccion,meses)
  base_perido_final = pd.merge(base_periodo, slope, left_on=['Cliente'], right_on=['Cliente'],how='left')
  
  name_slope='slope_'+str(meses)

  base_perido_final.rename(columns={'slope': name_slope}, inplace=True)

  return base_perido_final

In [5]:
def marge_bases(clientes_df,df_agg1,df_agg2,df_agg3,key,how):

        '''
    retorna la base train final 

    Parameters:
        clientes_df: Listado y caractaristicas por cliente.
        df_agg1: Base agrupada 1 (Largo plazo 12 meses).
        df_agg2 :Base agrupada 2 (Largo plazo 6 meses).
        df_agg3 :Base agrupada 3 (Largo plazo 3 meses).
        key : key de union
        how : how de union

    Returns:
        base_marge: Base final.
    '''

  base_marge = pd.merge(clientes_df, df_agg1, left_on=[key], right_on=[key],how=how)
  base_marge = pd.merge(base_marge, df_agg2, left_on=[key], right_on=[key],how=how)
  base_marge = pd.merge(base_marge, df_agg3, left_on=[key], right_on=[key],how=how)
  base_marge.fillna(0, inplace=True)
  base_marge.set_index('Cliente' , inplace = True)

  return base_marge

In [6]:
def base_target(df, fecha_prediccion):

        '''
    retorna la base target , con la variable a predecir segun la fecha de prediccion 

    Parameters:
        df: Listado y caractaristicas por cliente.
        fecha_prediccion:Fecha corte del mes que se desea predecir (Ejemplo : '2020-08-01')

    Returns:
        base_target: base target. '''

  base_target = df[df['Fecha']==pd.to_datetime(fecha_prediccion)]
  base_target = base_target[['Cliente','Marca']]
  base_target = pd.get_dummies(base_target)
  
  columns_t = base_target.columns.to_list()
  columns_t.remove('Cliente') 
  
  estadisticas_t = ['sum']
  
  agg_group_train ={}
  
  for i in columns_t :
    key=i
    agg_group_train[key] = estadisticas_t
  
  base_target = base_target.groupby(['Cliente']).agg(agg_group_train).reset_index()
  base_target.columns = ['_'.join(col) for col in base_target.columns]
  base_target.rename(columns={'Cliente_': 'Cliente'}, inplace=True)
  clientes=clientes_df_dummies[['Cliente']]
  base_target = pd.merge(clientes, base_target, left_on=['Cliente'], right_on=['Cliente'],how='left')
  base_target.fillna(0, inplace=True)
  base_target.set_index('Cliente' , inplace = True)

  base_target.rename(columns={'Marca_M_20-C_3-CE_9_sum': 'M_20-C_3-CE_9', 
                              'Marca_M_16-C_2-CE_10_sum':'M_16-C_2-CE_10',
                               'Marca_M_9-C_3-CE_12_sum':'M_9-C_3-CE_12',
                               'Marca_M_38-C_2-CE_10_sum':'M_38-C_2-CE_10',
                                'Marca_M_39-C_2-CE_10_sum':'M_39-C_2-CE_10'
                        }, inplace=True)

  return base_target

In [7]:
def scaler_X(df):
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(df)
  return X_scaled

# **Datasets** 

**input1_clientes_estructura.** = Tabla de clientes con las características de su punto de venta y ubicación.

*  Cliente : Id del Establecimiento (Llave primaria)
*  Regional2: Región donde se encuentra ubicado (Conjunto de departamentos)
*  Gerencia2: Gerencia que le distribuye el producto (Conjunto de 1, 2 o mas ciudades)
*  SubCanal2: Tipo de establecimiento: (Tienda, Bar, etc)
*  Categoria: Variable definida por AB-Inbev para calificar la importancia de un establecimiento dada su compra histórica.
*  Nevera: Variable dummie que me permite ver si el establecimiento tiene o no una nevera en su punto de venta.

**input2_clientes_venta** = Base de historia de compra por marca-cupo-capacidad_envase por establecimiento, de los últimos meses.

*   Año en el que se realiza la compra
*   Mes en el que se realiza la compra
*   Cliente: Id del establecimiento
*   SegmentoPrecio2: Categoría de la marca, dado el ingreso por litro a la compañía.
*   Marca2: Marca vendida.
*   Cupo2: Tipo de envase de la marca vendida.
*   CapacidadEnvase2: Capacidad en volumen, de la Marca2-Cupo2 vendido.
*   Volumen: Cantidad de volumen vendido de la Marca-Cupo2-CapacidadEnvase2.
*   disc: Descuentos otorgados al establecimiento por la compra
*   nr: NetRevenue que dejó la venta a la compañía.



In [33]:
# Bases
clientes_df = pd.read_csv("/content/drive/My Drive/Input1_clientes_estructura.csv", sep =';')
ventas_df = pd.read_csv("/content/drive/My Drive/Input2_clientes_venta.csv", sep =';')
base_pred= pd.read_csv("/content/drive/My Drive/Input3_clientes_test.csv", sep=";")

# **Data preparation** 

Con el fin de lograr el objetivo del reto, la preparación de datos se enfoco principalmente en la creación de variables enriquecidas buscando extrater la mayor cantidad de informacion de los datasets.

## Dataset de ventas

In [34]:
# Con el fin de facilitar el manejor de la variables se remplaza en las columas de Marca2, Cupo2 y CapacidadEnvase2 las palabras de Marca, Cupo y CapacidadEnvase2  por M , C Y CE.   
ventas_df.replace({'Marca2': r'^Marca','Cupo2': r'^Cupo','CapacidadEnvase2': r'^CapacidadEnvase'}
                  , {'Marca2': 'M', 'Cupo2':'C','CapacidadEnvase2':'CE'}, regex=True,inplace=True)

# A partir de las columnas de Año y Mes se crea una variable tipo fecha, esto para facilitar el desarrollo de agrupaciones dentro de la base.
ventas_df['Fecha'] = pd.to_datetime(ventas_df['Año'].astype(str)+ ventas_df['Mes'].apply(lambda x: '0'+str(x) if x<=9 else str(x)) +'01')

# A partir de las columnas de Marca2 , Cupo2, CapacidadEnvase2 se crea una varible denominada marca que hace referencia a un determinado producto.
ventas_df['Marca'] = ventas_df['Marca2']+'-'+ventas_df['Cupo2']+'-'+ventas_df['CapacidadEnvase2']

# Se borra las variables utilizadas anteriormente en la creacion de las variables enriquecidad iniciales.
ventas_df.drop(columns=['Año','Mes','Marca2','Cupo2','CapacidadEnvase2'], inplace=True)

In [59]:
# Se eliminan las marcas con apariciones dentro de la base menores a la mediana.
marcas_minimas =pd.DataFrame(ventas_df.groupby(['Marca'])['Marca'].count().sort_values())
marcas_minimas.columns=['Cantidad']
marcas_minimas.reset_index(inplace=True)
marcas= marcas_minimas[marcas_minimas['Cantidad']>=2098]['Marca'].to_list()
ventas_df=ventas_df[ventas_df['Marca'].isin(marcas)]

In [60]:
ventas_df.shape

(1381109, 8)

# Agrupacion de ventas mensuales por cliente.

Se determina a manera general el nivel de ventas que ha exitido por mes y por cliente.[link text](https://)

In [61]:
# Se detetermina por cliente y fecha la cantidad total de volumen, valor y los distintos productos.
grouped_cliente = ventas_df.groupby(['Cliente','Fecha']).agg({'Volumen': ['sum'],'disc': ['sum'],'nr': ['sum'],'Marca':['nunique']}).reset_index()

# Se renombra las columnas.
grouped_cliente.columns = ['Cliente','Fecha','volumen_cliente','descuento_cliente','valor_cliente','distinct_marca_cliente']

# Agrupacion de ventas mensuales por cliente y producto.

Se determina por producto el nivel de ventas por mes y cliente, de igual manera se establece la diferencia mensula de compra


In [62]:
# Se detetermina por cliente, producto  y fecha la cantidad total de volumen, valor.
grouped_cliente_marca = ventas_df.groupby(['Cliente','Fecha', 'Marca']).agg({'Volumen': ['sum'],'disc': ['sum'],'nr': ['sum']}).reset_index()

# Se renombra las columnas.
grouped_cliente_marca.columns = ['Cliente','Fecha','Marca','volumen_marca','descuento_marca','valor_marca']

# Se determina la diferencia del volumen con respescto al mes anterior.
grouped_cliente_marca.sort_values(['Cliente','Fecha','Marca'],inplace=True)
grouped_cliente_marca['diff_vol_marca']=grouped_cliente_marca.groupby(['Cliente','Marca'])['volumen_marca'].transform(pd.Series.diff).fillna(grouped_cliente_marca['volumen_marca'])
grouped_cliente_marca['ant_volumen_marca']=grouped_cliente_marca.groupby(['Cliente','Marca'])['volumen_marca'].shift(1)

## Ventas enriquecidas

Se crean variables enriquecidad con el fin de maximizar la extraccion de informacion del dataset de ventas.

In [63]:
# merge de los Dataframes contruidos anteriormente.

ventas_df_enr = pd.merge(grouped_cliente, grouped_cliente_marca, left_on=['Cliente','Fecha'], right_on=['Cliente','Fecha'], how="left")

In [64]:
# Construcción de variables enriquecidas.

# Porcentaje de volumen del producto por el total del cliente.
ventas_df_enr['p_vol'] =ventas_df_enr['volumen_marca']/ventas_df_enr['volumen_cliente']

# Porcentaje de descuento del producto por el total del cliente.
ventas_df_enr['p_desc'] =ventas_df_enr['descuento_marca']/ventas_df_enr['descuento_cliente']

# Porcentaje de valor del producto por el total del cliente.
ventas_df_enr['p_valor'] =ventas_df_enr['valor_marca']/ventas_df_enr['valor_cliente']

# Porcentaje de variacion del volumen de producto con respecto a mes anterior.
ventas_df_enr['p_variacion'] = (ventas_df_enr['diff_vol_marca']/ventas_df_enr['ant_volumen_marca']).apply(lambda x: 0 if x==math.inf else x)

# Se eliminan las variables utilizadas anteriormente.
ventas_df_enr.drop(columns=['volumen_marca','descuento_marca','valor_marca','diff_vol_marca','ant_volumen_marca'], inplace=True)

## One-hot encoding

Se crean varables por cada producto, ya que el consumo de un determinado producto puede estar influenciado por el consumo de los otros productos de portafolio. Tambien se crean variables dummies a parir de el dataset que contiene las caracteristicas de cada cliente.*italicized text* 

In [65]:
# Se hace un pivot sobre el dataframe ventas_df_enr, manteniendo los valores de Cliente ,Fecha,volumen_cliente,descuento_cliente,valor_cliente,distinct_marca_cliente
#Obteniendo una fila por cliente al mes.
ventas_df_encoding= pd.pivot_table(ventas_df_enr, values=['p_vol','p_desc','p_valor','p_variacion'], 
                                                  index=['Cliente','Fecha','volumen_cliente','descuento_cliente','valor_cliente','distinct_marca_cliente'],
                                                  columns=['Marca'], aggfunc=np.mean).reset_index()

#Se renobran las columas del dataframe.
ventas_df_encoding.columns = ['_'.join(col) for col in ventas_df_encoding.columns]
ventas_df_encoding.rename(columns={'Cliente_': 'Cliente', 
                        'Fecha_': 'Fecha',
                        'volumen_cliente_':'volumen_cliente',
                        'descuento_cliente_':'descuento_cliente',
                        'valor_cliente_':'valor_cliente',
                        'distinct_marca_cliente_':'distinct_marca_cliente'}, inplace=True)

# Los valores nulos se imputan con cero ya que si el valor es nulo el cliente no realizo la compra.
ventas_df_encoding.fillna(0, inplace=True)

In [66]:
# Creacion de variables dummies de clientes

clientes_df_dummies = pd.get_dummies(clientes_df)

## Agrupación de variables por periodos

Para caterizar el comporamiento de compra del cliente a corto, mediano y largo plazo se agrupo las variable en 3, 6, y 12 meses segun el mes que se quisiera predecir.


In [67]:
#Estadisticos de agrupación.
estadisticas = ['mean','median']

#Bases agrupadas por periodo para predecir agosto
ult_3_meses_ago= base_periodo(ventas_df_encoding,'2020-08-01',3, estadisticas)
ult_6_meses_ago= base_periodo(ventas_df_encoding,'2020-08-01',6, estadisticas)
ult_12_meses_ago= base_periodo(ventas_df_encoding,'2020-08-01',12, estadisticas)

#Bases agrupadas por periodo para predecir septiembre
ult_3_meses_sep= base_periodo(ventas_df_encoding,'2020-09-01',3, estadisticas)
ult_6_meses_sep= base_periodo(ventas_df_encoding,'2020-09-01',6, estadisticas)
ult_12_meses_sep= base_periodo(ventas_df_encoding,'2020-09-01',12, estadisticas)

/usr/local/lib/python3.6/dist-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/usr/local/lib/python3.6/dist-packages/scipy/stats/_stats_mstats_common.py:140: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/usr/local/lib/python3.6/dist-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [69]:
base_agos=marge_bases(clientes_df_dummies,ult_12_meses_ago,ult_6_meses_ago,ult_3_meses_ago,'Cliente','left')
base_sep=marge_bases(clientes_df_dummies,ult_12_meses_sep,ult_6_meses_sep,ult_3_meses_sep,'Cliente','left')

## Contrucción  Variable Target 

Se determina la compra o no del producto en el mes por cliente

In [70]:
target_ago = base_target(ventas_df_enr,'2020-08-01')
target_sep = base_target(ventas_df_enr,'2020-09-01')

## Data Modeling

Ya con las bases listas, se entrena el modelo que tiene el objetivo de determinar la probalidad que un cliente compre un producto en el siguiente mes.

In [71]:
# Lista de productos objetivo 
productos =["M_20-C_3-CE_9","M_16-C_2-CE_10","M_9-C_3-CE_12","M_38-C_2-CE_10","M_39-C_2-CE_10"]

In [ ]:
# Entrenamiento Modelo 

features_names = base_agos.columns.to_list()

#Se normalizan los valores 
X_scaled = scaler_X(base_agos)

# Modelo por cada producto objetivo

for i in productos :

  # Búsqueda por grid search con validación cruzada
  
  print('inicio ' + i)
  param_grid = {'n_estimators': [200],
              'max_features': [10,20,50],
              'max_depth'   : [ 3,  20],
              'criterion'   : ['entropy']
             }
  
  w = 50
  model = GridSearchCV(
        estimator  = RandomForestClassifier(class_weight={0: 1, 1: w},random_state = 9),
        param_grid = param_grid,
        scoring    = 'roc_auc',
        n_jobs     = multiprocessing.cpu_count() - 1,
        refit      = True,
        verbose    = 0,
        return_train_score = True)
  
  # Seleccion variable objetivo
  y_base=target_ago[i]
  
  # división de bases X y Y 
  print('division' + i)
  X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_scaled, y_base, test_size=0.3, random_state=8)
  
 # Entrenamiento
  print('modelo' + i)
  model.fit(X_train, Y_train)

  # AUC del modelo seleccionado por variable
  auc = metrics.roc_auc_score(Y_validation, model.predict_proba(X_validation)[:,1])
  print("AUC: " + str(i)+" "+ str(auc))
  print(model.best_params_)
  
  # Se guarda el modelo en un archivo pickle
  with open("/content/drive/My Drive/Reto/smodel_B{}.pickle".format(i), "wb") as f:
    pickle.dump(model, f)
  f.close()

##Validacion

Se valida el modelo con el siguiente mes.

In [ ]:
# Validacion de modelo

#Se normalizan los valores 

for i in productos: 
   with open("/content/drive/My Drive/Reto/smodel_B{}.pickle".format(i), "rb") as f:
     model = pickle.load(f)
     
     X_scaled = scaler_X(base_sep)
     
     predicted= model.predict(X_scaled)

     auc = roc_auc_score(target_sep[i], model.predict_proba(X_scaled)[:,1])
     print("AUC: " + str(i)+" "+ str(auc))

## output 

Se toma la base que debe predecir

In [ ]:
# Se extrae el listado de clientes 
clientes_filtro=base_pred['Cliente'].to_list()
clientes_pred= clientes_df_dummies[clientes_df_dummies["Cliente"].isin(clientes_filtro)]

# Se crea la base de variables

#Bases agrupadas por periodo para predecir agosto
ult_3_meses_oct= base_periodo(ventas_df_encoding,'2020-10-01',3, estadisticas)
ult_6_meses_oct= base_periodo(ventas_df_encoding,'2020-10-01',6, estadisticas)
ult_12_meses_oct= base_periodo(ventas_df_encoding,'2020-10-01',12, estadisticas)

#Base de predición
base_oct=marge_bases(clientes_pred,ult_12_meses_oct,ult_6_meses_oct,ult_3_meses_oct,'Cliente','left')

In [ ]:
#Predicción sobre base test

# Prediccion por producto objetivo
for i in productos: 
   with open("/content/drive/My Drive/Reto/smodel_B{}.pickle".format(i), "rb") as f:
     model = pickle.load(f)

     #Se normalizan los valores 
     base=base_oct[features_names]
     X_scaled = scaler_X(base)
     # Prediccion
     base_oct[i]=np.round(model.predict_proba(X_scaled)[:,1],3)
base_predic=base_oct[productos]

In [ ]:
# Se renombran las columna
base_predic.columns = ['Marca1',	'Marca2',	'Marca3',	'Marca_Inno1',	'Marca_Inno2']

#Se guarda el output
base_predic.to_csv("/content/drive/My Drive/Reto/outputB.csv")